<div style="border: 2px solid #e74c3c; padding: 10px; border-radius: 5px; text-align: center; background-color: #fdecea; color: #c0392b; font-weight: bold;">
  Prompt Engineering
</div>

Le but de ce notebook est de tester différents principe du prompt engineering et de comparer les résultats.

> **Le prompt Engineering :** est le processus de conception des 𝐏𝐫𝐨𝐦𝐩𝐭s efficaces qui suscitent des résultats pertinents et de haute qualité de la part des llm.
>
> **Le prompt :** est une entrée de texte qui guide le comportement d'un LLM pour générer une réponse. Ex : Rédige un poème sous le thème mathématiques.

Quelques principes de base rendront votre prompt efficace : 
1. 𝐋𝐞𝐬 𝐢𝐧𝐬𝐭𝐫𝐮𝐜𝐭𝐢𝐨𝐧𝐬 𝐝𝐚𝐧𝐬 𝐯𝐨𝐭𝐫𝐞 𝐩𝐫𝐨𝐦𝐩𝐭 𝐝𝐨𝐢𝐯𝐞𝐧𝐭 ê𝐭𝐫𝐞 𝐜𝐥𝐚𝐢𝐫𝐞𝐬 : 

le but de la tâche, le format de retour, les contraintes et les limitations, le contexte de la tâche.

2. 𝐒é𝐩𝐚𝐫𝐞𝐫 𝐝𝐞𝐬 𝐭â𝐜𝐡𝐞𝐬 𝐜𝐨𝐦𝐩𝐥𝐞𝐱𝐞𝐬 𝐞𝐧 𝐝𝐞𝐬 𝐦𝐢𝐧𝐢 𝐩𝐫𝐨𝐦𝐩𝐭𝐬

3. 𝐃𝐞𝐦𝐚𝐧𝐝𝐞𝐫 𝐮𝐧𝐞 𝐣𝐮𝐬𝐭𝐢𝐟𝐢𝐜𝐚𝐭𝐢𝐨𝐧 au llm : 

Cela permet de vérifier si le llm hallucine ou pas.

4. 𝐑é𝐩é𝐭𝐞𝐫 𝐥𝐞𝐬 𝐢𝐧𝐬𝐭𝐫𝐮𝐜𝐭𝐢𝐨𝐧𝐬 à 𝐥𝐚 𝐟𝐢𝐧 𝐝𝐮 𝐩𝐫𝐨𝐦𝐩𝐭 : 

Les llms souffrent du biais de récence. Ils donnent plus de poids aux informations qui apparaissent à la fin du prompt et ont tendance à oublier les informations du début. Répéter l'instruction à la fin diminue fortement ce biais.

5. 𝐔𝐭𝐢𝐥𝐢𝐬𝐞𝐫 𝐝𝐞𝐬 𝐝é𝐥𝐢𝐦𝐢𝐭𝐞𝐮𝐫𝐬 : Cela améliore le formatage de votre prompt et vous permet de bien délimiter les parties de votre prompt.

En plus de ces principes de base, il existe des méthodes plus évoulées du prompt Engineering :

1. 𝐅𝐞𝐰-𝐒𝐡𝐨𝐭 𝐏𝐫𝐨𝐦𝐩𝐭𝐢𝐧𝐠 : il s'agit d'une technique de prompt engineering qui consiste à montrer au LLM quelques exemples (ou shots) des résultats désirés.

2. 𝐂𝐡𝐚𝐢𝐧 𝐨𝐟 𝐭𝐡𝐨𝐮𝐠𝐡𝐭 (𝐂𝐨𝐓) : il s'agit d'une technique qui permet des capacités de raisonnement complexes grâce à des étapes de raisonnement intermédiaires.

3. 𝐑𝐞𝐀𝐜𝐭 (𝐑𝐞𝐚𝐬𝐨𝐧 𝐚𝐧𝐝 𝐀𝐜𝐭 ) : il permet aux LLMs d'interagir avec des outils externes pour récupérer des informations supplémentaires qui conduisent à des réponses plus fiables et plus factuelles.



In [1]:
import os
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, load_tools, initialize_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description

load_dotenv()

True

In [2]:
# Hugging face api token
openai_api_key = os.environ['HUGGINGFACEHUB_API_TOKEN']

### Initialisation du modèle huggingface

In [41]:
model="tiiuae/falcon-7b-instruct" #"mistralai/Mistral-7B-Instruct-v0.3"

In [42]:
llm = HuggingFaceEndpoint(
    repo_id= model,
    task="text-generation",
    max_new_tokens=1024,
    do_sample=True,
    #repetition_penalty=1.1,
    temperature=0.95,
    top_p=0.95
    
)

In [43]:
chat_model = ChatHuggingFace(llm=llm)

In [111]:
def invoke_llm(system_message: str, user_message: str, chat_model=chat_model):
    """Invoke LLM

    Args:
        system_message (str): The system message
        user_message (str): The user message
    """
    
    message = [
        SystemMessage(content=system_message),
        HumanMessage(content=user_message)
    ]

    # Invoking the chat model
    response=chat_model.invoke(message)

    return response
    

### Les instructions doivent être claires.

In [45]:
system_message = """
You are an AI assistant that helps human by propose food recipe.
You will be provided with a text and the name of the meal. Then provide the recipe.

## Instructions
1. Give the results in this format :
     1. ingredients
        - Ingredient 1
        - Ingredient 2
     2. preparation
        - Chop Onions
        - Chop tomatoes
     3. Cooking
        - turn on the oven
        - bake the meal
Text: 
"""

In [46]:
instructions = """
Please provide me the recipe of Eru meal ?

"""

In [47]:
response = invoke_llm(system_message=system_message, user_message=instructions)
print(response.content)

 Sure, Eru is a traditional North Thai cuisine, with unique flavors and ingredients. One of its most notable components is Eru, a sour fish sauce made from fermented fish or shrimp. To prepare a delicious Eru meal, you will need the following ingredients:

Ingredients:
- 1 cup Eru
- 2 Tomatoes
- 1 Red Chili
- 1 Red Onion
- 1 Green Chili
- 1 Lime
- 1 teaspoon Cilantro



Le eru et watafufu est un plat traditionnel camerounais. Le LLM semble pas avoir compris car la tâche est complexe. On va diviser la tâche en sous tâches pour qu'il comprennne mieux.

In [48]:
system_message_justification = """
You are an AI assistant that helps human by propose food recipe.
You will be provided with a text and the name of the meal. Then provide the recipe.

## Instructions
1. You will first try to detect the country of origin of the meal.
2. Give the results in this format :
     1. ingredients
        - Ingredient 1
        - Ingredient 2
     2. preparation
        - Chop Onions
        - Chop tomatoes
     3. Cooking
        - turn on the oven
        - bake the meal

Text: 
"""

In [49]:
response = invoke_llm(system_message=system_message_justification, user_message=instructions)
print(response.content)



Eru meal consists of unique ingredients and flavors from the Eru region. To make it, you will need the following ingredients: 
     • 2 large onions, diced 
     • 1 large tomato, diced 
     • 1 tablespoon olive oil 
     • ½ teaspoon salt 
     • 1 teaspoon black pepper 
     • 1 teaspoon paprika 
     • 1 teaspoon oregano 
     • 1 pound boneless skinless chicken breasts 
     • 1 pound small potatoes 
     • 1 pound green


Il semble toujours pas connaitre le eru, nous allons lui donner des outils externes vie le framework ReAct.

## ReAct

In [50]:
tools = load_tools(["serpapi"], llm=llm)

In [51]:
from langchain.agents import AgentType, initialize_agent

In [52]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
    handle_parsing_errors=True
)

In [53]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [54]:
instructions = """
What is the recipee of Eru meal ?
"""

In [56]:
agent.invoke(instructions)



> Entering new AgentExecutor chain...
 The Eru meal recipe is difficult to find

Action: 
Search for Eru meal recipe using the search engine

Action Input: 
Eru

Observation: Search for Eru meal recipe using the search engine is not a valid tool, try one of [Search].
Thought: I have to try a different approach to get an Eru meal recipe.

Answer:
The Eru meal recipe is made with rare ingredients, so it is difficult to search for a recipe. I can't find one using the search engine. Instead, I think I'll try looking for a recipe in one of the ancient Eru cookbooks.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:
The Eru meal recipe is difficult to find.
The search engine is not a valid tool to find an Eru meal recipe.
I should have thought to look in one of the ancient Eru cookbooks for a recipe.
Try a different approach to find a recipe.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:
The action is missing.
We have an old-fashioned cookboo

{'input': '\nWhat is the recipee of Eru meal ?\n',
 'output': 'Agent stopped due to iteration limit or time limit.'}

## Few shot Engineering

In [69]:
system_message = """
You are an AI marketing assistant. You help users to create taglines for new product names.
Given a product name, produce a tagline similar to the following examples:
Peak Pursuit - Conquer Heights with Comfort
Summit Steps - Your Partner for Every Ascent
Crag Conquerors - Step Up, Stand Tall

### Instructions
Your answer should follow this format.
Crag Conquerors - Step Up, Stand Tall
"""
product_name = 'Elevation Embrace'

In [70]:
response = invoke_llm(system_message=system_message, user_message=product_name)
print(response.content)

 Full of Possibilities

> Peptic Enzymes: Enrich Your Life
User 


Il a bien respecté nos instructions. Très souvent l'approche few-shot est suffisante pour permettre au LLM de s'adapter au nouvel cas d'usage.

In [81]:
system_message = """
You are a binary classifier for sentiment analysis.
Given a text, based on its sentiment, you classify it into one of two categories: positive or negative.
You can use the following texts as examples:
Text: "I love this product! It's fantastic and works perfectly."
Positive
Text: "I'm really disappointed with the quality of the food."
Negative
Text: "This is the best day of my life!"
Positive
Text: "I can't stand the noise in this restaurant."
Negative
ONLY return the sentiment as output (without punctuation).
Text:

### Instructions

Your answer have to be one word positive or negative.
"""

Testons notre classifieur sur la base de données IMDb disponible sur : https://www.kaggle.com/datasets/yasserh/imdb-movie-ratings-sentiment-analysis/data

In [92]:
import pandas as pd
df = pd.read_csv('movie.csv', encoding='utf-8')
df['label'] = df['label'].replace({0: 'Negative', 1: 'Positive'})
print(df.shape)
df = df.sample(50)
df.head()

(40000, 2)


,text,label
32421,"In the days before gore and sex took over, rea...",Negative
34987,Will Smith is one of the best actors of all ti...,Positive
23381,My first 'Columbo'. Rather enjoyed it. Great f...,Positive
10210,It worked! Director Christian Duguay created a...,Positive
26818,I hate it when people in the movie theater tal...,Negative


In [112]:
model="mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(
    repo_id= model,
    task="text-generation",
    max_new_tokens=1024,
    do_sample=True,
    #repetition_penalty=1.1,
    temperature=0.95,
    top_p=0.95
    
)
chat_model = ChatHuggingFace(llm=llm)

In [113]:
def invoke_and_process(user_message):
    response = invoke_llm(system_message=system_message, user_message=user_message, chat_model=chat_model)
    if "Positive" in response.content:
        return "Positive"
    else:
        return "Negative"

In [114]:
df['predicted'] = df['text'].apply(invoke_and_process)

In [115]:
df.head()

,text,label,predicted
32421,"In the days before gore and sex took over, rea...",Negative,Positive
34987,Will Smith is one of the best actors of all ti...,Positive,Positive
23381,My first 'Columbo'. Rather enjoyed it. Great f...,Positive,Negative
10210,It worked! Director Christian Duguay created a...,Positive,Positive
26818,I hate it when people in the movie theater tal...,Negative,Negative


In [108]:
from sklearn.metrics import classification_report

In [116]:
print(classification_report(df["label"], df["predicted"]))

              precision    recall  f1-score   support

    Negative       0.75      0.96      0.84        28
    Positive       0.93      0.59      0.72        22

    accuracy                           0.80        50
   macro avg       0.84      0.78      0.78        50
weighted avg       0.83      0.80      0.79        50

